In [3]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import pickle
from tqdm import tqdm
import re
import json
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from collections import Counter

print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [4]:
from model import *
from data import *
from utils import *

In [5]:
import os
os. environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda:0')

## Train

- **set params**

In [6]:
lr = 0.0001
batch_size=128

- **load train data**

In [7]:
news_vert = np.load('../../data2/news_vert.npy')
news_title = np.load('../../data2/news_title.npy')
news_body = np.load('../../data2/news_body.npy')

In [8]:
with open('../../data2/TrainUsers.pkl', 'rb') as f:
    TrainUsers = pickle.load(f)
with open('../../data2/TrainSamples.pkl', 'rb') as f:
    TrainSamples = pickle.load(f)

- **load model**

In [9]:
with open('../../data2/dict.pkl', 'rb') as f:
    _,category_dict,word_dict = pickle.load(f)
with open('../../data2/news.pkl', 'rb') as f:
    news = pickle.load(f)
embedding_matrix = np.load('../../data2/embedding_matrix.npy')

In [10]:
torch.cuda.empty_cache()

In [11]:
model = NRMS(embedding_matrix)
model = model.to(device)

- **begin training**

In [12]:
def acc(y_true, y_hat):
    y_hat = torch.argmax(y_hat, dim=-1)
    tot = y_true.shape[0]
    hit = torch.sum(y_true == y_hat)
    return hit.data.float() * 1.0 / tot

In [13]:
optimizer = optim.Adam(model.parameters(), lr=0.0001) #lr = 0.0001

In [14]:
min_train_loss = 100.0
for ep in range(1,4):
    loss = 0.0
    accuary = 0.0
    cnt = 1
    dset = TrainDataset(TrainUsers, TrainSamples, news_title, news_vert, news_body)
    data_loader = DataLoader(dset, batch_size=128, collate_fn=collate_fn, shuffle=True)
    tqdm_util = tqdm(data_loader)
    for user_feature, news_feature, label in tqdm_util: 
        user_feature = [i.to(device) for i in user_feature]
        news_feature = [i.to(device) for i in news_feature]
        label = label.to(device)
        bz_loss, y_hat = model(user_feature, news_feature, label)
        loss += bz_loss.data.float()
        accuary += acc(label, y_hat)

        optimizer.zero_grad()
        bz_loss.backward()
        optimizer.step()

        if cnt % 10 == 0:
            tqdm_util.set_description('ed: {}, train_loss: {:.5f}, acc: {:.5f}'.format(cnt * batch_size, loss.data / cnt, accuary / cnt))
        cnt += 1
    loss /= cnt
    print(ep, loss)
    torch.save(model.state_dict(), '../../runs/userencoder/NAML-{}.pkl'.format(ep))


  0%|          | 0/5963 [00:00<?, ?it/s]d:\Code\Fundamental_AI\PENS-Personalized-News-Headline-Generation\pensmodule\UserEncoder\data.py:66: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  user_feature = [torch.LongTensor(i) for i in zip(*user_feature)]
ed: 762880, train_loss: 0.59068, acc: 0.67883: 100%|██████████| 5963/5963 [30:30<00:00,  3.26it/s]


1 tensor(0.5906, device='cuda:0')


ed: 762880, train_loss: 0.55464, acc: 0.71287: 100%|██████████| 5963/5963 [30:09<00:00,  3.30it/s]


2 tensor(0.5545, device='cuda:0')


ed: 762880, train_loss: 0.54268, acc: 0.72221: 100%|██████████| 5963/5963 [30:23<00:00,  3.27it/s]


3 tensor(0.5426, device='cuda:0')


## Test

- **load test data**

In [15]:
with open('../../data2/ValidUsers.pkl', 'rb') as f:
    ValidUsers = pickle.load(f)
with open('../../data2/ValidSamples.pkl', 'rb') as f:
    ValidSamples = pickle.load(f)

In [16]:
for ep in range(1,4):
    model = NRMS(embedding_matrix)
    model = model.to(device)
    model.load_state_dict(torch.load('../../runs/userencoder/NAML-{}.pkl'.format(ep)))
    model.eval() 
    # save new embedding matrix
    np.save('../../data2/embedding_matrix{}.npy'.format(ep), model.embed.weight.data.cpu().numpy())
    

    n_dset = news_dataset(news_title, news_vert, news_body)
    news_data_loader = DataLoader(n_dset, batch_size=512, collate_fn=news_collate_fn, shuffle=False)

      
    news_scoring = []
    torch.cuda.empty_cache()
    with torch.no_grad():
        for news_feature in tqdm(news_data_loader): 
            news_feature = [i.to(device) for i in news_feature]
            news_vec = model.news_encoder(news_feature)
            news_vec = news_vec.to(torch.device("cpu")).detach().numpy()
            news_scoring.extend(news_vec)
    news_scoring = np.array(news_scoring)
    np.save('../../data2/news_scoring{}.npy'.format(ep), news_scoring)
    
    u_dset = UserDataset(news_scoring, ValidUsers)
    user_data_loader = DataLoader(u_dset, batch_size=128, shuffle=False)

    user_scoring = []
    with torch.no_grad():
        for user_feature in tqdm(user_data_loader): 
            user_feature = user_feature.to(device)
            user_vec = model.user_encoder(user_feature)
            user_vec = user_vec.to(torch.device("cpu")).detach().numpy()
            user_scoring.extend(user_vec)
    user_scoring = np.array(user_scoring)
    np.save('../../data2/global_user_embed{}.npy'.format(ep),np.mean(user_scoring,axis=0))
    g = evaluate(user_scoring,news_scoring, ValidSamples)
    print(ep)
    print('AUC\t', 'MRR\t', 'nDCG5\t', 'nDCG10\t','CTR1\t','CTR10\t')
    print(g)

100%|██████████| 100000/100000 [01:41<00:00, 982.72it/s]


0 0.0
1
AUC	 MRR	 nDCG5	 nDCG10	 CTR1	 CTR10	
(0.6436104504160457, 0.23353615763002544, 0.25621876444997116, 0.3346347817845591, 0.14899, 0.12019733333333334)


100%|██████████| 100000/100000 [01:37<00:00, 1023.63it/s]


0 0.0
2
AUC	 MRR	 nDCG5	 nDCG10	 CTR1	 CTR10	
(0.645814365689914, 0.23421902581878667, 0.2573015718911829, 0.3355176226438488, 0.14898, 0.12062533333333333)


100%|██████████| 100000/100000 [01:44<00:00, 958.61it/s]

0 0.0
3
AUC	 MRR	 nDCG5	 nDCG10	 CTR1	 CTR10	
(0.6491278531495707, 0.23887359880774814, 0.2636869254022473, 0.3417275924090409, 0.15598, 0.12316933333333333)
